## Day 30 Lecture 2 Assignment

In this assignment, we will learn about random forests. We will use the google play store dataset loaded below.

In [229]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [230]:
reviews = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/googleplaystore.csv')

reviews.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In this assignment, you will work more independently. Perform the following steps:
    
1. Select which columns are best suited to predict whether the rating is above 4.5
2. Process the data (including transforming to the correct column type, removing missing values, creating dummy variables, and removing irrelevant variables)
3. Create a random forest model and evaluate
4. Using grid search cross validation, tweak the parameters to produce a better performing model
5. Show and discuss your results

Good luck!

In [231]:
reviews = reviews.drop(['App','Type','Content Rating','Genres','Android Ver','Current Ver','Last Updated','Category'],axis=1)
reviews

,Rating,Reviews,Size,Installs,Price
0,4.1,159,19M,"10,000+",0
1,3.9,967,14M,"500,000+",0
2,4.7,87510,8.7M,"5,000,000+",0
3,4.5,215644,25M,"50,000,000+",0
4,4.3,967,2.8M,"100,000+",0
...,...,...,...,...,...
10836,4.5,38,53M,"5,000+",0
10837,5.0,4,3.6M,100+,0
10838,NaN,3,9.5M,"1,000+",0
10839,4.5,114,Varies with device,"1,000+",0


In [232]:
#adding new col
reviews = reviews.dropna()
reviews['Elite'] = reviews['Rating'].map(lambda x: 1 if x>4.5 else 0)
reviews = reviews.drop('Rating',axis=1)
reviews

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Reviews,Size,Installs,Price,Elite
0,159,19M,"10,000+",0,0
1,967,14M,"500,000+",0,0
2,87510,8.7M,"5,000,000+",0,1
3,215644,25M,"50,000,000+",0,0
4,967,2.8M,"100,000+",0,0
...,...,...,...,...,...
10834,7,2.6M,500+,0,0
10836,38,53M,"5,000+",0,0
10837,4,3.6M,100+,0,1
10839,114,Varies with device,"1,000+",0,0


In [233]:
#adjusting size step 1
reviews = reviews[reviews['Size'] != 'Varies with device']
reviews

,Reviews,Size,Installs,Price,Elite
0,159,19M,"10,000+",0,0
1,967,14M,"500,000+",0,0
2,87510,8.7M,"5,000,000+",0,1
3,215644,25M,"50,000,000+",0,0
4,967,2.8M,"100,000+",0,0
...,...,...,...,...,...
10833,44,619k,"1,000+",0,1
10834,7,2.6M,500+,0,0
10836,38,53M,"5,000+",0,0
10837,4,3.6M,100+,0,1


In [234]:
#adjusting install
reviews = reviews[reviews['Installs'] != 'Free']
reviews['Installs'] = reviews['Installs'].map(lambda x: int(x.rstrip("+").replace(',', '')))
reviews

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Reviews,Size,Installs,Price,Elite
0,159,19M,10000,0,0
1,967,14M,500000,0,0
2,87510,8.7M,5000000,0,1
3,215644,25M,50000000,0,0
4,967,2.8M,100000,0,0
...,...,...,...,...,...
10833,44,619k,1000,0,1
10834,7,2.6M,500,0,0
10836,38,53M,5000,0,0
10837,4,3.6M,100,0,1


In [235]:
#adjusting Price 
reviews['Price'] = reviews['Price'].map(lambda x: int(x.lstrip("$").replace('.', '')))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [236]:
#adjusting size
reviews['Size'] = reviews['Size'].map(lambda x: x.replace('M','') + '000000' if x != x.replace('M','') else x)
reviews['Size'] = reviews['Size'].map(lambda x: x.replace('k','') + '000' if x != x.replace('k','') else x)
reviews['Size'] = reviews['Size'].map(lambda x: x.replace('.','')[:-1] if x != x.replace('.','') else x)
reviews['Size'] = reviews['Size'].map(lambda x: int(x))

reviews.dtypes

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Reviews     object
Size         int64
Installs     int64
Price        int64
Elite        int64
dtype: object

In [237]:
reviews['Reviews'] = reviews['Reviews'].map(lambda x: x.replace('M','') + '000000' if x != x.replace('M','') else x)
reviews['Reviews'] = reviews['Reviews'].map(lambda x: x.replace('.','')[:-1] if x != x.replace('.','') else x)
reviews['Reviews'] = reviews['Reviews'].map(lambda x: int(x))
reviews.dtypes

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

Reviews     int64
Size        int64
Installs    int64
Price       int64
Elite       int64
dtype: object

In [238]:
reviews

,Reviews,Size,Installs,Price,Elite
0,159,19000000,10000,0,0
1,967,14000000,500000,0,0
2,87510,8700000,5000000,0,1
3,215644,25000000,50000000,0,0
4,967,2800000,100000,0,0
...,...,...,...,...,...
10833,44,619000,1000,0,1
10834,7,2600000,500,0,0
10836,38,53000000,5000,0,0
10837,4,3600000,100,0,1


In this assignment, you will work more independently. Perform the following steps:
    
1. Select which columns are best suited to predict whether the rating is above 4.5
2. Process the data (including transforming to the correct column type, removing missing values, creating dummy variables, and removing irrelevant variables)
3. Create a random forest model and evaluate
4. Using grid search cross validation, tweak the parameters to produce a better performing model
5. Show and discuss your results

Good luck!

In [239]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
X = reviews.drop('Elite',axis=1)
y= reviews['Elite']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [240]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [241]:
rf.score(X_train,y_train)

0.9972505256348051

In [242]:
rf.score(X_test,y_test)

0.796895213454075

In [263]:
grid = {'n_estimators':[50,60,70,80,90,100,110,120]}

from sklearn.model_selection import GridSearchCV

result = GridSearchCV(RandomForestClassifier(), grid, cv=5)
result.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [267]:
print('best number of estimator: ',result.best_estimator_.n_estimators)
print('best score: ',result.best_score_)

best number of estimator:  120
best score:  0.8030071971807772
